In [4]:
#Caricare le librerie
import tweepy
import pandas as pd
import numpy as np
import re


In [2]:
consumer_key= "rrR0Znz98oKkjfB8b72oLH9Qc"
consumer_secret= "m7iOPX6iW8R3IalRf7x56YVeIb18o3I1bhUftg8JzlQ6fAjLgW"
access_token= "1442250607-3cJYKYyh7Ae19nMcv5YoxOIB2XlvgVX0ROaDSJx"
access_token_secret= "ZOnaaCTgO7DsKtmPjVruOlkvzBY8p6MbkKLjiYLdamBYE"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [5]:
#Mostra i miei follower
for user in tweepy.Cursor(api.get_followers, screen_name="davide_pstars").items():
  print('follower: ' + user.screen_name)

follower: kimilfung
follower: werbond
follower: a____capo
follower: SalsaConAndrea


In [6]:
#Mostra chi seguo (amici)
#Nel metodo items si può inserire un numero intero, a signifcare il numero di risultati da restituire.
for user in tweepy.Cursor(api.get_friends, screen_name="@davide_pstars").items():
  print('friend: ' + user.screen_name)

friend: reportrai3


In [7]:
# Mostro la mia timeline 
stuff = api.user_timeline(screen_name = '@davide_pstars', count = 10)

In [5]:
#Scaricare in un dataframe i tweet in base ad un hashtag (until="2021-12-21")
hashtag="#fedez"
list_tweet=[]
for tweet in tweepy.Cursor(api.search_tweets,q=hashtag,count=100,lang="it").items(100):
  #print(tweet.text)
  #print('')
  list_status=np.where(hasattr(tweet, "retweeted_status"),True,False)
  list_tweet.append([tweet.created_at, tweet.id, tweet.text, 
                     tweet.favorite_count, tweet.retweet_count, 
                    tweet.user.location,tweet.user.screen_name,list_status])
  
tweets = pd.DataFrame(list_tweet[1:], 
                      columns=['date','id','text','like','n_rt','location','author',"retweet"])
tweets.head()

,date,id,text,like,n_rt,location,author,retweet
0,2021-12-18 21:20:40+00:00,1472315904230924296,"RT @rtl1025: 📌 Anche #Fedez e #ChiaraFerragni,...",0,4,,Bianca34874951,True
1,2021-12-18 21:14:28+00:00,1472314343358443530,"Quando vuoi passare per furba, ma ti dimostri ...",0,0,,ElisabettaZano7,False
2,2021-12-18 21:12:50+00:00,1472313933004423175,@Fedez: Ci ritroveremo sempre qua.\nChiara che...,0,0,,Vera220302,False
3,2021-12-18 20:59:51+00:00,1472310666824400896,"@redazionerumors Ovvio, ha il delirio di onnip...",0,0,,ElisabettaZano7,False
4,2021-12-18 20:49:45+00:00,1472308122865881088,RT @IlContiAndrea: E comunque la nonna di #Fed...,0,4,"Italy , Rome",flamanc24,True


Object `remove` not found.
